In [ ]:
import pygame
from pygame.locals import *  
import sys
import random
import numpy as np
import pygame
from pygame.locals import *  
import sys
import random
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pickle
tf.test.gpu_device_name()

In [ ]:
tf.test.gpu_device_name()

# Flappy Bird Game

In [ ]:
class Bird:
    def __init__(self):
        #initialize with starting coordinates, yspeed, and r, g, b for color
        self.xPos=float(150)
        self.yPos=float(random.randint(100,700))
        self.ySpeed=float(0)
        self.r=random.randint(0,255)
        self.g=random.randint(0,255)
        self.b=random.randint(0,255)
    def drawBird(self):
        #draw bird on pygame screen
        pygame.draw.rect(screen, (self.r,self.g,self.b), [self.xPos,self.yPos,20,20])
    def jump(self):
        #function for when bird jumps
        self.ySpeed=-8
    def drag(self):
        #effect of gravity on bird
        self.ySpeed+=.4
    def move(self):
        #let yspeed effect position of bird
        self.yPos+=self.ySpeed
    def checkbounds(self):
        #stop play if bird is out of bounds
        if self.yPos>800:
            play=False
            
class Pillar:
    def __init__(self, i):
        self.i=i
        self.speed=float(-3)
        self.xPos=float(500.0+(i*200))
        self.opening=float(random.randint(100,700))
        self.scorecounted= False
    def drawPillar(self):
        pygame.draw.rect(screen, black, [self.xPos-10, 0, 20, self.opening-65])
        pygame.draw.rect(screen, black, [self.xPos-10, self.opening+65, 20, 800-self.opening-65])
    def move(self):
        self.xPos+=self.speed
    def checkPosition(self):
        global score
        if self.xPos<0:
            self.xPos+=600
            self.opening=random.randint(100,700)
            self.scorecounted=False
        if self.xPos<150 and self.scorecounted==False:
            self.scorecounted=True
            score=score+1
        
screen= pygame.display.set_mode((500, 800))
black=(0,0,0)
white=(255,255,255)
gameExit=False
p=[]
b=Bird()
play=True
score=0
p.append(Pillar(0))
p.append(Pillar(1))
p.append(Pillar(2))

refresh_count=0
while not gameExit:
    refresh_count=refresh_count+1
    if refresh_count>80000:
        refresh_count=0
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                gameExit=True
            if (event.type == pygame.KEYDOWN) and play==True:
                b.jump()
        screen.fill(white)
        b.drawBird()
        b.move()
        b.drag()
        for i in range(3):
            p[i].drawPillar()
            p[i].checkPosition()
            p[i].move()
        pygame.font.init()
        font = pygame.font.SysFont("Arial", 50)
        screen.blit(font.render(str(score), -1, (0, 0, 0)), (200, 50))
        #collision detection and boundaries
        for i in range(3): 
            if (b.xPos<p[i].xPos+15 and b.xPos>p[i].xPos-15) and (b.yPos<p[i].opening-65 or b.yPos>p[i].opening+65-15):
                play=False
        if not 0 < b.yPos < 800:
            play=True
            score=0
            b.yPos=400
            for i in range(3):
                p[i].scorecounted=False
            p[0].xPos=500
            p[1].xPos=700
            p[2].xPos=900

        pygame.display.update()

# TensorFlow NeuroEvolution

In [ ]:
class NeuralNetwork:
    def __init__(self, a, b, c):
        if isinstance(a, NeuralNetwork):
            self.model=a
        else:
            self.input=a
            self.hidden=b
            self.output=c
            self.model=tf.keras.Sequential([
                tf.keras.layers.Dense(self.hidden, input_shape=(self.input,), activation='sigmoid'),
                tf.keras.layers.Dense(self.output, activation='softmax'),
            ])
    def predict(self, inputs):
        output=self.model.predict(inputs)
        return output[0]
    def randomweights(self):
        for j, layer in enumerate(self.model.layers):
            newweightsforlayer = []
            #loop through each matrix, there are 2 (connection weights and biases)
            for weightarray in layer.get_weights():
                #save their shape
                saveshape = weightarray.shape
                #reshape them to one dimension
                onedimweight = weightarray.reshape(-1)
                for i, weight in enumerate(onedimweight):
                    #give them random weights between -1,1
                    onedimweight[i]= random.uniform(-1,1)
                #reshape them back to the original form
                newweightarray = onedimweight.reshape(saveshape)
                #save them to the weight list for the layer
                newweightsforlayer.append(newweightarray)
            #set the new weight list for each layer
            self.model.layers[j].set_weights(newweightsforlayer)
    def mutate(self):
        for j, layer in enumerate(self.model.layers):
            newweightsforlayer = []
            #loop through each matrix, there are 2 (connection weights and biases)
            for weightarray in layer.get_weights():
                #save their shape
                saveshape = weightarray.shape
                #reshape them to one dimension
                onedimweight = weightarray.reshape(-1)
                for i, weight in enumerate(onedimweight):
                    #mutate based on rate of 15%
                    if random.random() <= .15:
                        x=onedimweight[i]+np.random.normal(0,.1)
                        onedimweight[i]=x
                #reshape them back to the original form
                newweightarray = onedimweight.reshape(saveshape)
                #save them to the weight list for the layer
                newweightsforlayer.append(newweightarray)
            #set the new weight list for each layer
            self.model.layers[j].set_weights(newweightsforlayer)


class Bird:
    def __init__(self, a):
        self.xPos=float(150)
        self.yPos=float(random.randint(250,550))
        self.ySpeed=float(0)
        self.r=random.randint(0,255)
        self.g=random.randint(0,255)
        self.b=random.randint(0,255)
        self.score=float(0)
        self.fitness=None
        self.hasjumped=False
        self.counter=0
        if isinstance(a, NeuralNetwork):
            self.brain=a
        else:
            self.brain=NeuralNetwork(5,8,2)
    def drawBird(self):
        pygame.draw.rect(screen, (self.r,self.g,self.b), [self.xPos,self.yPos,20,20])
    def jump(self):
        self.ySpeed=-8
    def mutate(self):
        self.brain.mutate()
    def think(self,p):
        closestp=None
        closestd=10000
        for i in range(3):
            d=p[i].xPos-self.xPos
            if d<closestd and d>0:
                closestp=p[i]
                closestd=d
        self.inputs=np.array([[self.yPos/800, self.xPos/500, (closestp.xPos-100)/500, (closestp.opening-100)/800, (closestp.opening+100)/800]])
        self.output=self.brain.predict(self.inputs)
        if self.output[0]>self.output[1] and self.hasjumped==False:
            self.jump()
            self.hasjumped=True
            self.counter=0
    def drag(self):
        self.ySpeed+=.4
    def move(self):
        self.yPos+=self.ySpeed
        if self.hasjumped==True and self.counter>=18:
            self.hasjumped=False      

class Pillar:
    def __init__(self, i):
        self.i=i
        self.speed=float(-3)
        self.xPos=float(500.0+(i*300))
        self.opening=float(random.randint(200,600))
        self.scorecounted= False
    def drawPillar(self):
        pygame.draw.rect(screen, black, [self.xPos-10, 0, 20, self.opening-65])
        pygame.draw.rect(screen, black, [self.xPos-10, self.opening+65, 20, 800-self.opening-65])
    def move(self):
        self.xPos+=self.speed
    def checkPosition(self):
        global score
        if self.xPos<0:
            self.xPos+=900
            self.opening=random.randint(200,600)
            self.scorecounted=False
        if self.xPos<150 and self.scorecounted==False:
            self.scorecounted=True
            score=score+1
            
            
screen= pygame.display.set_mode((500, 800))
black=(0,0,0)
white=(255,255,255)
gameExit=False
p=[]
b=[]
deadb=[]
deadbfitness=[]
score=0
p.append(Pillar(0))
p.append(Pillar(1))
p.append(Pillar(2))
zaz=None
gen=1
#cycles=25
pressed=False
cyclecount=0
hassavedbird=False
for i in range(1):
    b.append(Bird(zaz))

def pickbird():
    i=0
    r=random.random()
    while r>0:
        r=r-deadb[i].fitness
        i=i+1
    i=i-1
    birdchild=Bird(deadb[i].brain)
    birdchild.mutate()
    return birdchild

def nextgenreg():
    #calculate fitness 
    #run genreg when at least 320 birds get through the first pillar
    s=0
    for i in range(len(deadb)):
        s=s+deadb[i].score
    for i in range(len(deadb)):
        deadb[i].fitness=deadb[i].score/s
    #pickone and add to list b(alive birds) for each bird in deadb   
    for i in range(len(deadb)):
        b.append(pickbird())
    #empty dead birds and startover
    deadb.clear()
    
def nextgengood1(): 
    #calculate fitness 
    #runs when only one bird gets through
    #creates new list with best bird and mutates best bird
    s=0
    for i in range(len(deadb)):
        s=s+deadb[i].score
    for i in range(len(deadb)):
        deadb[i].fitness=deadb[i].score/s
    #find best bird based on fitness
    bestb=deadb[0]
    for i in range(len(deadb)):
        if deadb[i].score>bestb.score:
            bestb=deadb[i]
    birdchild=Bird(bestb.brain)
    for i in range(len(deadb)):
        birdchild.mutate()
        b.append(birdchild)
    deadb.clear()
    
def nextgengood2():
    #runs when at least two birds get through the first pillar
    #add them into a new list, and then based on their fitness you select them into the next generation
    goodbirds=[]
    for i in range(len(deadb)):
        if deadb[i].score>118:
            goodbirds.append(deadb[i])
    #calculate fitness 
    s=0
    for i in range(len(goodbirds)):
        s=s+goodbirds[i].score
    for i in range(len(goodbirds)):
        goodbirds[i].fitness=goodbirds[i].score/s
    for i in range(len(deadb)):
        z=0
        r=random.random()
        while r>0:
            r=r-goodbirds[z].fitness
            z=z+1
        z=z-1
        birdchild=Bird(goodbirds[z].brain)
        birdchild.mutate()
        b.append(birdchild)
    deadb.clear()
    
    
while not gameExit:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            gameExit=True
    ########################
    #drawing stuff
    screen.fill(white)
    pygame.font.init()
    font1 = pygame.font.SysFont("Arial", 50)
    screen.blit(font1.render(str(gen), -1, (0, 0, 0)), (20, 40))
    font = pygame.font.SysFont("Arial", 50)
    screen.blit(font.render(str(score), -1, (0, 0, 0)), (200, 50))
    for i in range(len(b)):
        b[i].drawBird()
    for i in range(3):
            p[i].drawPillar()
    ########################
    if len(b)!=0:
        for i in range(len(b)):
            b[i].score=b[i].score+1
            b[i].move()
            b[i].drag()
            b[i].think(p)
            b[i].counter=b[i].counter+1
        for i in range(3):
            p[i].checkPosition()
            p[i].move()
        #collision detection and boundaries
        for j in range(len(b)-1,-1,-1):
            for i in range(3): 
                if j<=len(b)-1:
                    if not 0 < b[j].yPos < 800:
                        deadb.append(b.pop(j))
                if j<=len(b)-1:
                    if (b[j].xPos<p[i].xPos+10 and b[j].xPos>p[i].xPos-15) and (b[j].yPos<p[i].opening-65 or b[j].yPos>p[i].opening+65-15):
                        deadb.append(b.pop(j))
    if len(b)==1 and b[0].score>=3000*117 and hassavedbird==False:
        savedbird=b[0].brain.model.get_weights()
        file_bird=open('bestbird3', 'wb')
        pickle.dump(savedbird, file_bird)
        file_bird.close()
        hassavedbird=True
    #make next gen of birds
    if len(b)==0:
        gen=gen+1
        for i in range(3):
            p[i].scorecounted=False
            p[i].opening=float(random.randint(200,600))
        p[0].xPos=500
        p[1].xPos=800
        p[2].xPos=1100
        trashbcounter=0
        goodbcounter=0
        for i in range(len(deadb)):
            if deadb[i].score<=120:
                trashbcounter=trashbcounter+1
            if deadb[i].score>118:
                goodbcounter=goodbcounter+1
        if score==0:
            nextgenreg()
            for i in range(len(b)):
                b[i].brain.randomweights()
            print("ran randomweights")
        else:
            if goodbcounter==1:
                nextgengood1()
                print("ran nextgengood 1")
            else:
                nextgengood2()
                print("ran nextgengood 2")
        score=0
    pygame.display.update()
